In [6]:
!python.exe -m pip install --upgrade pip

In [105]:
!pip install pytesseract
!pip install beautifulsoup4
!pip install pandas
# !pip install tensorflow
!pip install Pillow
!pip install lxml
!pip install opencv-python
!pip install pathlib

In [106]:
from bs4 import BeautifulSoup
import base64
import pandas as pd
# import tensorflow as tf
from pytesseract import pytesseract
from PIL import Image
import glob
import cv2
import numpy as np
from pathlib import Path

In [101]:
# %cd C:\Users\user\Desktop\Working\向高手求救--落點分析交叉查榜依考區查榜

# *交叉查榜*.html

In [113]:
%cd C:\Users\User\Desktop\backup\brt\Projects\Crawing-List-of-Admitted-Schools-for-Students\data

for file in glob.iglob('*交叉查榜*.html'):
    # print(file)
    processing_department(file)

C:\Users\User\Desktop\backup\brt\Projects\Crawing-List-of-Admitted-Schools-for-Students\data
正1

111*x**0

正3

112"xx*2

正……

110xx%*2

正7

110x«x*8

正9

110xxx*9

正11

110xxx*9

正13

110x«x*5

正1……

112xxx*4

正17

110xx%*7

備1

110*xx*0

備3

L11xxx*5

備5

111xxx*8

備7

110xxx*3

備9

110xx%*4

備11

110xx%*4

備13

111xxx*8

備15

L11xxx*5

備 17

L11xxx*7

備 19

111*x**6

備21

110xx%*2

備23

112*xx*3

備25

L11xxx*2

110x«x*8

110xx%*4

110*xx*0

110xx%*7

L1lxxx«1

L1lxxx«1

112*x**0

正2

L11xxx*3

正乙ˍ

112*xx*8

正6

110xx%*4

正8

L1lxxx«1

正1〔)

110xxx*9

正12

110xxx*9

正1乙ˍ

110x«x*5

正16

110xxx*3

正18

113%x%*7

備2

L11xxx*3

備4

110xxx*9

備6

L11xxx*7

備8

110xx%*4

備10

111xxx*8

備 12

111*xx*9

備 M

110xx%*7

備16

110*x**6

備18

110xxx*3

備20

111*xx*9

備22

L11xxx*3

備24

L1lxxx«1

110x«x*8

110xxx*3

110xxx*3

110xx%*2

111xxx*8

L11xxx*3

112*xx*3



In [112]:
def processing_department(filename):
    
    with open(filename, 'r', encoding="utf-8") as file:
        html = file.read()
        # print(html)
        soup = BeautifulSoup(html, 'html.parser')
    
    fields = ['rank', 'exam_location', 'stuno', 'univ',  'year']
    df = pd.DataFrame(columns=fields)
    df1 = pd.DataFrame(columns=fields)
    df2 = pd.DataFrame(columns=fields)

    str = soup.head.find_all('title')[0].text
    pos = str.find('年')
    year = str[pos-3:pos]  # str[5:8] str[5], str[6], str[7]

    str = soup.head.find_all('title')[0].text
    pos = str.find('-')
    univ = str[:pos].rstrip()

    dark_list = soup.body.find_all(bgcolor="#DEDEDC") # 褐色
    df1 = processing_list(univ, year, dark_list, df)
    # print(df1)
    
    white_list = soup.body.find_all(bgcolor="#FFFFFF") # 白色
    df2 = processing_list(univ, year, white_list, df)
    # print(df2)
    
    # craw more data
    
    # make two df cross-interleave
    max_len = max(len(df1), len(df2))
    for i in range(max_len):
        if i < len(df1):
            df = pd.concat([df, df1.iloc[[i]]], ignore_index=True)
        if i < len(df2):
            df = pd.concat([df, df2.iloc[[i]]], ignore_index=True)

    result_dir = Path(r"C:\Users\User\Desktop\backup\brt\Projects\Crawing-List-of-Admitted-Schools-for-Students\result",f'{univ}.csv')
    df.to_csv(path_or_buf = result_dir, encoding='utf-8-sig', index=False)

In [109]:
def processing_list(univ, year, list, df):
    # print(list) # html
    #print(df)
    for dark_item in list:
        x = dark_item.children # get message under the soup.body.find_all(bgcolor="")
        count = 0
        for i in x:
            # print("i:",i)
            if count == 3:
                # print("i:")
                # print(i.text)  #正1
                # rank = i.text
                try:
                    base64_str = i.find_all('img')[0]['src'].split(',')[1]
                except IndexError:
                    # print("not exist")
                    rank = "None"
                else:
                    with open("rank.png", "wb") as fh:
                        fh.write(base64.decodebytes(bytes(base64_str, 'utf-8')))
                    path_to_tesseract = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
                    path_to_image = r'rank.png'
                    pytesseract.tesseract_cmd = path_to_tesseract
                    thresholding_(r'rank.png')
                    path_to_image = r'rank.png'
                    rank = pytesseract.image_to_string(Image.open(path_to_image), lang='chi_tra', config='--oem 0 --psm 7')
                    print(rank)
            elif count == 5:
                #print(i.find_all('img')[0]['src'].split(',')[1])
                base64_str = i.find_all('img')[0]['src'].split(',')[1]
                with open("num.png", "wb") as fh:
                    fh.write(base64.decodebytes(bytes(base64_str, 'utf-8')))
                path_to_tesseract = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
                pytesseract.tesseract_cmd = path_to_tesseract
                thresholding_(r'num.png')
                path_to_image = r'num.png'
                stuno = pytesseract.image_to_string(Image.open(path_to_image), lang='eng', config='--oem 1 --psm 7')
                print(stuno) # 應試號碼(num)
            count += 1
    
        x = dark_item.find_all('a')
        # print(x[0].text[6:]) # 大學
        exam_location = x[0].text[6:] 
        #ptr_df = ptr_df + 1
    
        dict = {'rank': rank, 'exam_location': exam_location, 'stuno': stuno, 'univ': univ, 'year': year}
        df = pd.concat([df, pd.DataFrame([dict])], ignore_index = True)
    
    return df

In [108]:
def thresholding_(png):
    if png[0:3] == "num":
        image = cv2.imread(png)
        gray = get_grayscale(image)
        thresh_ = thresholding(gray)
        cv2.imwrite('num.png', thresh_)
    else:
        image = cv2.imread(png)
        gray = get_grayscale(image)
        thresh_ = thresholding(gray)
        cv2.imwrite('rank.png', thresh_)        
  

# cv2

In [74]:
# Read the image
img = cv2.imread(r'C:\Users\User\Desktop\backup\brt\Projects\Crawing-List-of-Admitted-Schools-for-Students\data\rank.png')

In [107]:
# Convert the image to grayscale
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Remove noise from the image
def remove_noise(image):
    return cv2.medianBlur(image, 5)
 
# Thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

# Dilation
def dilate(image):
    kernel = np.ones((5, 5), np.uint8)
    return cv2.dilate(image, kernel, iterations=1)
    
# Erosion
def erode(image):
    kernel = np.ones((5, 5), np.uint8)
    return cv2.erode(image, kernel, iterations=1)

# Opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5, 5), np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

# Canny edge detection
def canny(image):
    return cv2.Canny(image, 100, 200)

# Skew correction
def deskew(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]

    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

# Template matching
def match_template(image, template):
    return cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)

In [76]:
gray = get_grayscale(img)
thresh = thresholding(gray)
opening = opening(gray)
canny = canny(gray)

In [80]:
cv2.imwrite('My Image.png', thresh)


True